In [ ]:
# The following is to know when this notebook has been run and with which python version.
import time, sys
print(time.ctime())
print(sys.version.split('|')[0])

# G The astropy package

The Astropy Project is a community effort to develop a single core package for Astronomy in Python and foster interoperability between Python astronomy packages. More informations here: http://www.astropy.org/

https://www.slideshare.net/KelleCruz/astropy-project-update-for-adass

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### Constants and Units

http://docs.astropy.org/en/stable/constants/index.html

http://docs.astropy.org/en/stable/units/index.html

In [ ]:
import astropy
print(astropy.__version__)
from astropy import constants as const
from astropy import units as u
help(const)

In [ ]:
# Pretty printing
print(const.c)

In [ ]:
# .to change the unit
print(const.c.to('Mpc/yr'))

In [ ]:
# basic operations are managed
const.c ** 2

In [ ]:
np.sqrt(const.c)

In [ ]:
print(np.sqrt(const.c))

In [ ]:
# Following the units
M1 = 3 * const.M_sun
M2 = 100 * u.g
Dist = 2.2 * u.au
F = const.G * M1 * M2 / Dist ** 2
print(M1)
print(F)

In [ ]:
F

In [ ]:
# Convert in more classical unit
print(F.to(u.N))

In [ ]:
q = 42.0 * u.meter

In [ ]:
q**2

In [ ]:
# Extract only the value
print((q**2).value)
print(q.value**2)

In [ ]:
arr = np.array([q.value, q.value]) * const.G
print(type(arr))
print(arr)

In [ ]:
arr2 = np.ones(2) * q
arr2[1] = q*3
arr2

In [ ]:
arr = np.ones(2) * q * const.G
print(type(arr))
print(arr)

In [ ]:
# Resolving redondant units
t = 3.0 * u.kilometer / (130.51 * u.meter / u.second)
print(t)
print(t.decompose())

In [ ]:
x = 1.0 * u.parsec
print(x.to(u.km))

In [ ]:
lam = 5007 * u.angstrom

In [ ]:
print(lam.to(u.nm))
print(lam.to(u.micron))

In [ ]:
# Some transformations needs extra information, available from u.special
print(lam.to(u.GHz, equivalencies=u.spectral()))

More in http://docs.astropy.org/en/stable/units/index.html

### Data Table

http://docs.astropy.org/en/stable/table/index.html

In [ ]:
from astropy.table import Table

In [ ]:
# create a table with non homogeneous types
a = [1, 4, 5]
b = [2.0, 5.0, 8.2]
c = ['x', 'y', 'z']
t = Table([a, b, c], names=('a', 'b', 'c'), meta={'name': 'first table'})
print(t)

In [ ]:
# Pretty output
t

In [ ]:
# One can change the output format
t['b'].format = '7.3f'
t['a'].format = '{:.4f}'
# and add units
t['b'].unit = 's'
t

In [ ]:
t.show_in_browser(jsviewer=True)

In [ ]:
# access the column names
t.colnames

In [ ]:
# length of the table (number of rows)
len(t)

In [ ]:
# Acces one element
t['a'][1]

In [ ]:
# Modify one element
t['a'][1] = 10
t

In [ ]:
# easy add column:
t['d'] = [1, 2, 3]

In [ ]:
t

In [ ]:
t.rename_column('a', 'A')
t

In [ ]:
t.add_row([-6.6, -9.3, 'r', 10])
t

In [ ]:
t.add_row([-9, 40, 'q', 10])
t

In [ ]:
# Masked values
t = Table([a, b, c], names=('a', 'b', 'c'), masked=True)
t['a'].mask = [True, True, False] # True is for the masked values!!
t

In [ ]:
t['a'].mask = [True, False, False] # True is for the masked values!!
t

In [ ]:
# Creat a table from a table
t2 = Table([t['a']**2, t['b']**2, t['a']**2 + t['b']**2], names=('a2', 'b2', 'a2+b2'))
t2

In [ ]:
# Managing columns
from astropy.table import Column

In [ ]:
# Create a table combining different formats
a = (1, 4)
b = np.array([[2, 5, 4, 5,6, 3], [5,  7, 6]])  # vector column
c = Column(['x', 'y'], name='axis')
f, (ax1, ax2) = plt.subplots(1,2)
d = Column([ax1, ax2], name='axis obj')
tup = (a, b, c, d)
t3 = Table(tup)  # Data column named "c" has a name "axis" in that table
t3

In [ ]:
# table from a dictionnary
rr = {'a': [1, 4],
      'b': [2.0, 5.0],
      'c': ('x', 'y')}
t4 = Table(rr)
t4

In [ ]:
# Create table row by row
t5 = Table(rows=[{'a': 5, 'b': 10}, {'a': 15, 'b': 30}])
t5

In [ ]:
# Numpy structured array
arr = np.array([(1, 2.0, 'x'),
                (4, 5.0, 'y')],
               dtype=[('a', 'i8'), ('b', 'f8'), ('c', 'S2')])
print(arr)
t6 = Table(arr)
print(t6)

Python arrays versus `numpy` arrays as input

There is a slightly subtle issue that is important to understand in the way that Table objects are created. Any data input that looks like a Python list (including a tuple) is considered to be a list of columns. In contrast an homogeneous numpy array input is interpreted as a list of rows:

In [ ]:
t7 = Table(((1,2,3), (4,5,6), (7,8,9)))
t7

In [ ]:
arr7 = np.array(((1,2,3), (4,5,6)))
t7 = Table(arr7)
print(arr7)
print(t7)

In [ ]:
arr = np.array([(1, 2.0, 'x'),
                (4, 5.0, 'y')],
               dtype=[('a', 'i8'), ('b', 'f8'), ('c', 'S2')])
t6 = Table(arr, copy=False) # pointing to the original data
arr['a'][0] = 99
print(arr)
print(t6)

In [ ]:
t6.columns

In [ ]:
t6.colnames

In [ ]:
# One can obtain a numpy structured array from a Table
np.array(t6)

In [ ]:
arr = np.arange(3000).reshape(100, 30)  # 100 rows x 30 columns array
t = Table(arr)
print(t)

In [ ]:
t.show_in_browser(jsviewer=True)

In [ ]:
# create a simple table to play with
arr = np.arange(15).reshape(5, 3)
t = Table(arr, names=('a', 'b', 'c'), meta={'keywords': {'key1': 'val1'}})
t

In [ ]:
t['a'] = [1, -2, 3, -4, 5]  # Set all
t

In [ ]:
t['a'][2] = 30 # set one
t

In [ ]:
# set one row
t[1] = (8, 9, 10)
t

In [ ]:
# Set a whole column
t['a'] = 99
t

In [ ]:
# Add a column
t.add_column(Column(np.array([1,2,3,4,5]), name='d'))
t

In [ ]:
# remove a column
t.remove_column('b')
t

In [ ]:
# add a row
t.add_row([-8, -9, 10])
t

In [ ]:
# Remove some rows
t.remove_rows([1, 2])
t

In [ ]:
# sort the Table using one column
t.sort('c')
t

In [ ]:
filter = (t['a'] > 50) & (t['d'] > 3)
print(filter)

In [ ]:
t[filter]

In [ ]:
%%writefile tab1.dat
#name    obs_date    mag_b  mag_v
M31     2012-01-02  17.0   17.5
M31     2012-01-02  17.1   17.4
M101    2012-01-02  15.1   13.5
M82     2012-02-14  16.2   14.5
M31     2012-02-14  16.9   17.3
M82     2012-02-14  15.2   15.5
M101    2012-02-14  15.0   13.6
M82     2012-03-26  15.7   16.5
M101    2012-03-26  15.1   13.5
M101    2012-03-26  14.8   14.3

In [ ]:
# directly read a Table from an ascii file
obs = Table.read('tab1.dat', format='ascii')

In [ ]:
print(obs)

In [ ]:
# Group data
obs_by_name = obs.group_by('name')
obs_by_name

In [ ]:
print(obs_by_name.groups.keys)

In [ ]:
# Using 2 keys to group
print(obs.group_by(['name', 'obs_date']).groups.keys)

In [ ]:
# Extracting a group
print(obs_by_name.groups[1])

In [ ]:
# Using a mask to select entries
mask = obs_by_name.groups.keys['name'] == 'M101'
print(mask)
print(obs_by_name.groups[mask])

In [ ]:
# Some functions can be applied to the elements of a group
obs_mean = obs_by_name.groups.aggregate(np.mean) 
print(obs_mean)

In [ ]:
print(obs_by_name['name', 'mag_v', 'mag_b'].groups.aggregate(np.mean))

In [ ]:
# creat a new Table on the fly
obs1 = Table.read("""name    obs_date    mag_b  logLx
M31     2012-01-02  17.0   42.5
M82     2012-10-29  16.2   43.5
M101    2012-10-31  15.1   44.5""", format='ascii')

In [ ]:
# this is used to stack Tables
from astropy.table import vstack

In [ ]:
tvs = vstack([obs, obs1])
tvs

In [ ]:
%%writefile data6.dat
Line        Iobs     lambda  rel_er   Obs_code
H  1  4861A 1.00000    4861. 0.08000  Anabel                               
H  1  6563A 2.8667     6563. 0.19467  Anabel                               
H  1  4340A 0.4933     4340. 0.03307  Anabel                               
H  1  4102A 0.2907     4102. 0.02229  Anabel                               
H  1  3970A 0.1800     3970. 0.01253  Anabel                               
N  2  6584A 2.1681     6584. 0.08686  Anabel                               
N  2 121.7m 0.0044621217000. 0.20000  Liu                                  
O  1  6300A 0.0147     6300. 0.00325  Anabel                               
TOTL  2326A 0.07900    2326. 0.20000  Adams                                
C  2 157.6m 0.00856 1576000. 0.20000  Liu                                  
O  1 63.17m 0.13647  631700. 0.10000  Liu                                  
O  1 145.5m 0.00446 1455000. 0.200    Liu                                  
TOTL  3727A 0.77609    3727. 0.200    Torres-Peimbert                      
S II  4070A 0.06174    4070. 0.200    Torres-Peimbert                      
S II  4078A 0.06174    4078. 0.200    Torres-Peimbert                      

In [ ]:
d = Table.read('data6.dat', format='ascii.fixed_width', 
               col_starts=(0, 12, 20, 29, 38))
d

In [ ]:
d.group_by('Obs_code')

There is a lot of possibilities of joining Tables, see http://docs.astropy.org/en/stable/table/operations.html

In [ ]:
t = Table.read("ftp://cdsarc.u-strasbg.fr/pub/cats/J/other/RMxAA/45.261/digeda.dat", 
               format='ascii.cds', 
               readme='ftp://cdsarc.u-strasbg.fr/pub/cats/J/other/RMxAA/45.261/ReadMe')

In [ ]:
t

In [ ]:
t.show_in_browser(jsviewer=True)

In [ ]:
plt.scatter(np.log10(t['I6583']), np.log10(t['I5007']), c=t['RefN'], edgecolor='None')
plt.xlabel(r'[NII]/H$\alpha$')
plt.ylabel(r'log [OIII]/H$\beta$')

In [ ]:
t = Table.read("ftp://cdsarc.u-strasbg.fr/pub/cats/VII/253/snrs.dat",
readme="ftp://cdsarc.u-strasbg.fr/pub/cats/VII/253/ReadMe",
format="ascii.cds")

In [ ]:
t

In [ ]:
t.show_in_browser(jsviewer=True)

In [ ]:
t[0:10].write('tab_cds1.tex', format='latex', overwrite=True, formats={'Sp-Index': '%0.2f'})

In [ ]:
!cat tab_cds1.tex

In [ ]:
t[10:20].write('tab_cds1.ascii', format='ascii', delimiter='|', formats={'Sp-Index': '%0.2f'}, overwrite=True)

In [ ]:
!cat tab_cds1.ascii

In [ ]:
t[10:20].write('tab_cds2.ascii', format='ascii.fixed_width', delimiter='', formats={'Sp-Index': '%0.2f'}, overwrite=True)

In [ ]:
!cat tab_cds2.ascii

The astropy Table can also read FITS files (if containing tables), VO tables and hdf5 format. See more there: http://docs.astropy.org/en/stable/io/unified.html

### Time and Dates

The astropy.time package provides functionality for manipulating times and dates. Specific emphasis is placed on supporting time scales (e.g. UTC, TAI, UT1, TDB) and time representations (e.g. JD, MJD, ISO 8601) that are used in astronomy and required to calculate, e.g., sidereal times and barycentric corrections. It uses Cython to wrap the C language ERFA time and calendar routines, using a fast and memory efficient vectorization scheme. More here: http://docs.astropy.org/en/stable/time/index.html

### Coordinates

The coordinates package provides classes for representing a variety of celestial/spatial coordinates, as well as tools for converting between common coordinate systems in a uniform way.

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord

In [ ]:
c = SkyCoord(ra=10.5*u.degree, dec=41.2*u.degree, frame='icrs')
c

In [ ]:
c = SkyCoord('0 42 00 +41 12 00', frame='icrs', unit=(u.hourangle, u.deg))
c

In [ ]:
print(c.ra, c.dec)

In [ ]:
c.to_string('decimal')

In [ ]:
print(c.dec.to_string(format='latex'))

$$41^\circ12{}^\prime00{}^{\prime\prime}$$

### Modeling

astropy.modeling provides a framework for representing models and performing model evaluation and fitting. It currently supports 1-D and 2-D models and fitting with parameter constraints.

It is designed to be easily extensible and flexible. Models do not reference fitting algorithms explicitly and new fitting algorithms may be added without changing the existing models (though not all models can be used with all fitting algorithms due to constraints such as model linearity).

The goal is to eventually provide a rich toolset of models and fitters such that most users will not need to define new model classes, nor special purpose fitting routines (while making it reasonably easy to do when necessary).

http://docs.astropy.org/en/stable/modeling/index.html

### Convolution and filtering

astropy.convolution provides convolution functions and kernels that offers improvements compared to the scipy scipy.ndimage convolution routines, including:

* Proper treatment of NaN values
* A single function for 1-D, 2-D, and 3-D convolution
* Improved options for the treatment of edges
* Both direct and Fast Fourier Transform (FFT) versions
* Built-in kernels that are commonly used in Astronomy

More on http://docs.astropy.org/en/stable/convolution/index.html